In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.image as img
from matplotlib import font_manager, rc
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name)

import seaborn as sns
sns.set(font="Malgun Gothic",rc={"axes.unicode_minus":False},
       style="darkgrid")

import cv2 as cv

import os

import glob

from openpyxl import load_workbook

from datetime import datetime

In [4]:
df=pd.read_csv("c:/data/dacon/open/total_with_yellow_f.csv")

In [6]:
df

,시간,내부온도관측치,외부온도관측치,내부습도관측치,외부습도관측치,CO2관측치,EC관측치,최근분무량,화이트 LED동작강도,레드 LED동작강도,...,난방부하,총추정광량,백색광추정광량,적색광추정광량,청색광추정광량,case,image_area_green,image_area_yellow,무게,월일
0,2021-05-28 02:46,22.600000,21.299999,78.199997,54.000000,597.0,0.0,126.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,75,126668.0,3549.0,210.872,05-28
1,2021-05-28 02:47,22.600000,21.400000,79.099998,54.000000,598.0,0.0,126.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,75,126668.0,3549.0,210.872,05-28
2,2021-05-28 02:48,22.700001,21.400000,79.599998,54.000000,607.0,0.0,126.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,75,126668.0,3549.0,210.872,05-28
3,2021-05-28 02:49,22.700001,21.299999,79.900002,54.000000,608.0,0.0,126.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,75,126668.0,3549.0,210.872,05-28
4,2021-05-28 02:50,22.700001,21.400000,80.300003,54.000000,610.0,0.0,126.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,75,126668.0,3549.0,210.872,05-28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2292678,2021-05-29 19:35,23.799999,22.799999,83.900002,52.900002,552.0,0.0,126.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,75,127746.0,3092.5,209.824,05-29
2292679,2021-05-29 19:40,23.900000,23.000000,81.900002,53.099998,539.0,0.0,126.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,75,127746.0,3092.5,209.824,05-29
2292680,2021-05-29 19:41,23.799999,22.900000,83.300003,52.900002,541.0,0.0,126.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,75,127746.0,3092.5,209.824,05-29
2292681,2021-05-29 19:42,23.500000,22.799999,81.000000,52.799999,543.0,0.0,126.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,75,127746.0,3092.5,209.824,05-29


In [7]:
# 시간 -> 월일 컬럼, case-> case_idx
df['월일']=df['시간'].str[5:10]
df['case_idx']=df['case']

# 시간, case 컬럼 제거
df.drop(['시간','case'],axis=1,inplace=True)

# meta 없는 case 제거
df_1=df.groupby("case_idx").max()
idx_notnull=df_1[df_1['최근분무량'].notnull()].index
# print(idx_notnull)
# len(idx_notnull)
df_1=df[df['case_idx'].isin(idx_notnull)]
df_1.head(5)

,내부온도관측치,외부온도관측치,내부습도관측치,외부습도관측치,CO2관측치,EC관측치,최근분무량,화이트 LED동작강도,레드 LED동작강도,블루 LED동작강도,...,난방부하,총추정광량,백색광추정광량,적색광추정광량,청색광추정광량,image_area_green,image_area_yellow,무게,월일,case_idx
0,22.600000,21.299999,78.199997,54.0,597.0,0.0,126.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,126668.0,3549.0,210.872,05-28,75
1,22.600000,21.400000,79.099998,54.0,598.0,0.0,126.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,126668.0,3549.0,210.872,05-28,75
2,22.700001,21.400000,79.599998,54.0,607.0,0.0,126.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,126668.0,3549.0,210.872,05-28,75
3,22.700001,21.299999,79.900002,54.0,608.0,0.0,126.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,126668.0,3549.0,210.872,05-28,75
4,22.700001,21.400000,80.300003,54.0,610.0,0.0,126.0,0.0,0.0,NaN,...,0.0,0.0,0.0,0.0,0.0,126668.0,3549.0,210.872,05-28,75


In [8]:
# case_idx, 월일 별 mean
df_2=df_1.groupby(['월일','case_idx']).mean()
df_2=df_2.reset_index()
df_2=df_2.sort_values(['case_idx','월일'])
df_2=df_2.reset_index()
df_2.drop('index',axis=1,inplace=True)
df_2.head()

,월일,case_idx,내부온도관측치,외부온도관측치,내부습도관측치,외부습도관측치,CO2관측치,EC관측치,최근분무량,화이트 LED동작강도,...,냉방부하,난방온도,난방부하,총추정광량,백색광추정광량,적색광추정광량,청색광추정광량,image_area_green,image_area_yellow,무게
0,05-07,1,22.809498,43.402276,77.435950,5.352071,475.723705,20.044034,0.0,200.672104,...,202.837487,19.356107,10.363331,164.957756,14.009672,23.870059,NaN,31914.500000,8.000000,49.193000
1,05-08,1,22.962222,43.638194,77.654236,4.840069,478.774306,20.661806,0.0,200.856836,...,179.413149,18.852110,13.461307,145.929643,12.389964,21.093542,NaN,37719.604861,13.750694,58.545399
2,05-09,1,22.926528,40.362361,77.490139,4.615646,485.095139,20.682222,0.0,200.856120,...,179.554817,18.854995,13.325044,146.041083,12.375941,21.137793,NaN,44738.948611,24.674306,70.774368
3,05-10,1,21.050139,56.355903,79.911042,4.006066,488.493056,18.266180,0.0,200.422495,...,179.504056,18.854097,7.632850,146.006121,12.370097,21.127837,NaN,52421.771528,41.040278,84.177522
4,05-11,1,21.818680,67.788264,81.271111,4.686528,483.207639,19.295069,0.0,200.422302,...,179.523273,18.854948,10.663438,145.999566,12.396976,21.126731,NaN,60250.952778,11.150000,100.600333


In [10]:
# case_idx, 월일 별 max
df_max=df_1.groupby(['월일','case_idx']).max()
df_max=df_max.reset_index()
df_max=df_max.sort_values(['case_idx','월일'])
df_max=df_max.reset_index()
df_max.drop(['index','image_area_green','image_area_yellow','무게'],axis=1,inplace=True)
df_max.head()

,월일,case_idx,내부온도관측치,외부온도관측치,내부습도관측치,외부습도관측치,CO2관측치,EC관측치,최근분무량,화이트 LED동작강도,레드 LED동작강도,블루 LED동작강도,냉방온도,냉방부하,난방온도,난방부하,총추정광량,백색광추정광량,적색광추정광량,청색광추정광량
0,05-07,1,29.400000,66.400002,87.800003,201.0,598.0,23.900000,0.0,201.0,201.0,201.0,24.0,401.9447,26.0,30.296585,309.41,47.9892,61.0935,NaN
1,05-08,1,30.900000,57.900002,87.699997,201.0,600.0,27.299999,0.0,201.0,201.0,201.0,24.0,401.9447,26.0,43.824081,309.41,47.9892,61.0935,NaN
2,05-09,1,30.700001,56.700001,89.199997,201.0,620.0,25.600000,0.0,201.0,201.0,201.0,24.0,401.9447,26.0,39.953247,309.41,47.9892,61.0935,NaN
3,05-10,1,26.100000,69.099998,89.900002,201.0,634.0,19.799999,0.0,201.0,201.0,201.0,24.0,401.9447,26.0,19.086292,309.41,47.9892,61.0935,NaN
4,05-11,1,29.000000,76.900002,89.599998,201.0,627.0,23.500000,0.0,201.0,201.0,201.0,24.0,401.9447,26.0,33.594360,309.41,47.9892,61.0935,NaN


In [11]:
# case_idx, 월일 별 min
df_min=df_1.groupby(['월일','case_idx']).min()
df_min=df_min.reset_index()
df_min=df_min.sort_values(['case_idx','월일'])
df_min=df_min.reset_index()
df_min.drop(['index','image_area_green','image_area_yellow','무게'],axis=1,inplace=True)
df_min.head()

,월일,case_idx,내부온도관측치,외부온도관측치,내부습도관측치,외부습도관측치,CO2관측치,EC관측치,최근분무량,화이트 LED동작강도,레드 LED동작강도,블루 LED동작강도,냉방온도,냉방부하,난방온도,난방부하,총추정광량,백색광추정광량,적색광추정광량,청색광추정광량
0,05-07,1,17.700001,28.799999,72.099998,0.0,360.0,16.700001,0.0,0.0,201.0,0.0,14.0,0.0,15.0,0.0,0.0,0.0,0.0,NaN
1,05-08,1,16.500000,28.799999,54.200001,0.0,347.0,14.500000,0.0,1.0,1.0,0.0,14.0,0.0,15.0,0.0,0.0,0.0,0.0,NaN
2,05-09,1,17.600000,23.000000,42.799999,0.0,345.0,15.800000,0.0,0.0,0.0,0.0,14.0,0.0,15.0,0.0,0.0,0.0,0.0,NaN
3,05-10,1,17.900000,43.500000,60.900002,0.0,346.0,17.000000,0.0,0.0,0.0,0.0,14.0,0.0,15.0,0.0,0.0,0.0,0.0,NaN
4,05-11,1,17.900000,54.900002,66.400002,0.0,341.0,17.400000,0.0,0.0,0.0,0.0,14.0,0.0,15.0,0.0,0.0,0.0,0.0,NaN


In [12]:
#df_msx, df_min 컬럼 이름 변경
df_max.columns=['월일', 'case_idx', '내부온도관측치_max', '외부온도관측치_max', '내부습도관측치_max', '외부습도관측치_max', 'CO2관측치_max',
       'EC관측치_max', '최근분무량_max', '화이트 LED동작강도_max', '레드 LED동작강도_max', '블루 LED동작강도_max', '냉방온도_max',
       '냉방부하_max', '난방온도_max', '난방부하_max', '총추정광량_max', '백색광추정광량_max', '적색광추정광량_max', '청색광추정광량_max']
df_max.head()

df_min.columns=['월일', 'case_idx', '내부온도관측치_min', '외부온도관측치_min', '내부습도관측치_min', '외부습도관측치_min', 'CO2관측치_min',
       'EC관측치_min', '최근분무량_min', '화이트 LED동작강도_min', '레드 LED동작강도_min', '블루 LED동작강도_min', '냉방온도_min',
       '냉방부하_min', '난방온도_min', '난방부하_min', '총추정광량_min', '백색광추정광량_min', '적색광추정광량_min', '청색광추정광량_min']
df_min.head()

,월일,case_idx,내부온도관측치_min,외부온도관측치_min,내부습도관측치_min,외부습도관측치_min,CO2관측치_min,EC관측치_min,최근분무량_min,화이트 LED동작강도_min,레드 LED동작강도_min,블루 LED동작강도_min,냉방온도_min,냉방부하_min,난방온도_min,난방부하_min,총추정광량_min,백색광추정광량_min,적색광추정광량_min,청색광추정광량_min
0,05-07,1,17.700001,28.799999,72.099998,0.0,360.0,16.700001,0.0,0.0,201.0,0.0,14.0,0.0,15.0,0.0,0.0,0.0,0.0,NaN
1,05-08,1,16.500000,28.799999,54.200001,0.0,347.0,14.500000,0.0,1.0,1.0,0.0,14.0,0.0,15.0,0.0,0.0,0.0,0.0,NaN
2,05-09,1,17.600000,23.000000,42.799999,0.0,345.0,15.800000,0.0,0.0,0.0,0.0,14.0,0.0,15.0,0.0,0.0,0.0,0.0,NaN
3,05-10,1,17.900000,43.500000,60.900002,0.0,346.0,17.000000,0.0,0.0,0.0,0.0,14.0,0.0,15.0,0.0,0.0,0.0,0.0,NaN
4,05-11,1,17.900000,54.900002,66.400002,0.0,341.0,17.400000,0.0,0.0,0.0,0.0,14.0,0.0,15.0,0.0,0.0,0.0,0.0,NaN


In [13]:
# mean, max, min df 월일과 case_idx 기준 병합
result_1=pd.merge(df_max,df_min,on=['case_idx','월일'])
result=pd.merge(df_2,result_1,on=['case_idx','월일'])
result.head(5)
result.columns

Index(['월일', 'case_idx', '내부온도관측치', '외부온도관측치', '내부습도관측치', '외부습도관측치', 'CO2관측치',
       'EC관측치', '최근분무량', '화이트 LED동작강도', '레드 LED동작강도', '블루 LED동작강도', '냉방온도',
       '냉방부하', '난방온도', '난방부하', '총추정광량', '백색광추정광량', '적색광추정광량', '청색광추정광량',
       'image_area_green', 'image_area_yellow', '무게', '내부온도관측치_max',
       '외부온도관측치_max', '내부습도관측치_max', '외부습도관측치_max', 'CO2관측치_max', 'EC관측치_max',
       '최근분무량_max', '화이트 LED동작강도_max', '레드 LED동작강도_max', '블루 LED동작강도_max',
       '냉방온도_max', '냉방부하_max', '난방온도_max', '난방부하_max', '총추정광량_max',
       '백색광추정광량_max', '적색광추정광량_max', '청색광추정광량_max', '내부온도관측치_min',
       '외부온도관측치_min', '내부습도관측치_min', '외부습도관측치_min', 'CO2관측치_min', 'EC관측치_min',
       '최근분무량_min', '화이트 LED동작강도_min', '레드 LED동작강도_min', '블루 LED동작강도_min',
       '냉방온도_min', '냉방부하_min', '난방온도_min', '난방부하_min', '총추정광량_min',
       '백색광추정광량_min', '적색광추정광량_min', '청색광추정광량_min'],
      dtype='object')

In [14]:
# 내보내기
result.to_csv("C:/data/dacon/open/result_3m_meta_yellow.csv", index = False)